# Prompt + LLM

In [8]:
!pip install --upgrade --quiet langchain langchain-core langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 11.8 MB/s eta 0:00:00


In [2]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-**********************************************"

## PromptTemplate + LLM

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()
chain = prompt | model

chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

### Attaching Stop Sequences

In [4]:
chain = prompt | model.bind(stop=["\n"])

chain.invoke({"foo": "bears"})

AIMessage(content='Why did the bear dissolve in water?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

### Attaching Function Call information

In [5]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

chain.invoke({"foo": "bears"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"setup":"Why don\'t bears like fast food?","punchline":"Because they can\'t catch it!"}', 'name': 'joke'}}, response_metadata={'finish_reason': 'stop', 'logprobs': None})

## PromptTemplate + LLM + OutputParser

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

chain.invoke({"foo": "bears"})

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!"

### Functions Output Parser

In [9]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

chain.invoke({"foo": "bears"})

{'setup': "Why don't bears like fast food?",
 'punchline': "Because they can't catch it!"}

In [10]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke({"foo": "bears"})

"Why don't bears like fast food?"

## Simplifying input

In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(foo=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke("bears")

"Why don't bears wear shoes?"

In [12]:
chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke("bears")

"Why don't bears like fast food?"